In [1]:
import sqlalchemy
import psycopg2
import os
import getpass

In [2]:
#Static Name for this project set by the 'Database_Pusher_Accidents_and_State_Data' notebook
database_name = 'bootcampfinalproject_accidents'

In [3]:
print('Enter AWS Endpoint:')
endpoint = getpass.getpass()

Enter AWS Endpoint:
········


In [4]:
print('Enter AWS Database Password: ')
aws_pass = getpass.getpass()

Enter AWS Database Password: 
········


In [5]:
#Should There be a different username set for your AWS Database, change this variable.
postgres = 'postgres'

In [6]:
def connect_db(passw = aws_pass, endp = endpoint, db = database_name, pg = postgres):
    #Connect to new database
    try:        
        db_string = f"postgresql+psycopg2://postgres:{passw}@{endpoint}:5432/{db}"
        engine = sqlalchemy.create_engine(db_string)
        print(f'Connected to Database: {db}')
    except Exception as exc:
        print(f'FAILED TO CONNECT to Database: {db}')
        print(exc)
    
    return engine

In [7]:
engine = connect_db()

Connected to Database: bootcampfinalproject_accidents


In [8]:
def get_tables():
    query = "SELECT * FROM information_schema.tables WHERE table_schema = 'public';"
    with engine.connect() as con:
        return con.execution_options(stream_results = True).execute(query).all()

In [9]:
all_tables_available = [i[2] for i in get_tables()]
print('Pulling Table Names from the AWS Database:')
all_tables_available

Pulling Table Names from the AWS Database:


['all_us_interstates',
 'blue_laws',
 'all_us_highways',
 'state_population_data_july2020',
 'US_Accidents_Dec20_Updated',
 'weed_legalization_raw_scraped',
 'data_grouped_by_state',
 'land_area_by_state_scraped',
 'road_distance_by_state',
 'DatabaseJoin_Deliverable']

# Targets

In [10]:
with engine.connect() as con:
    
    for i in all_tables_available:

        output = con.execution_options(stream_results = True).execute(
            f'SELECT * FROM "{i}" LIMIT 1').keys()
        print(i)
        for j in output:
            print('\t' + j)
    print('\n')

all_us_interstates
	full_name
	state_name
	road_geom
blue_laws
	State
	blueLaws
all_us_highways
	full_name
	state_name
	road_geom
state_population_data_july2020
	SUMLEV
	REGION
	DIVISION
	STATE
	NAME
	SEX
	ORIGIN
	RACE
	AGE
	CENSUS2010POP
	POPESTIMATE2019
US_Accidents_Dec20_Updated
	ID
	Severity
	Start_Time
	End_Time
	Start_Lat
	Start_Lng
	End_Lat
	End_Lng
	Distance(mi)
	Description
	Number
	Street
	Side
	City
	County
	State
	Zipcode
	Country
	Timezone
	Airport_Code
	Weather_Timestamp
	Temperature(F)
	Wind_Chill(F)
	Humidity(%)
	Pressure(in)
	Visibility(mi)
	Wind_Direction
	Wind_Speed(mph)
	Precipitation(in)
	Weather_Condition
	Amenity
	Bump
	Crossing
	Give_Way
	Junction
	No_Exit
	Railway
	Roundabout
	Station
	Stop
	Traffic_Calming
	Traffic_Signal
	Turning_Loop
	Sunrise_Sunset
	Civil_Twilight
	Nautical_Twilight
	Astronomical_Twilight
weed_legalization_raw_scraped
	State
	Legal Status
	Medicinal
	Decriminalized
	State Laws
data_grouped_by_state
	State
	CrashCount_2017
	CrashCount_2018
	

# Joins

state_population_data_july2020 on NAME <br>
blue_laws on State <br>
land_area_by_state_scraped on State <br>
road_distance_by_state on state_name <br>
weed_legalization_raw_scraped on State <br>

In [11]:
import pandas as pd

In [12]:
with engine.connect() as con:
    output = con.execution_options(stream_results = True).execute(
        '''
        SELECT 
            "land"."State",
            "land"."Total_SqMi",
            "land"."Total_SqKm",
            "land"."LandArea_SqMi",
            "land"."LandArea_SqKm",
            "blue"."blueLaws",
            "weed"."Legal Status" AS "WeedLegalStatus",
            "weed"."Medicinal" AS "MedicinalWeed",
            "weed"."Decriminalized" AS "WeedDecriminalized",
            "road"."sum_interstate_distance",
            "road"."sum_highway_distance",
            "road"."sum_other_distance"
        FROM "land_area_by_state_scraped" AS land 
        INNER JOIN blue_laws as blue
        ON (blue."State" = land."State")
        LEFT JOIN weed_legalization_raw_scraped AS weed
        ON (weed."State" = land."State")
        LEFT JOIN road_distance_by_state as road
        ON (road."state_name" = land."State")
        ORDER BY land."State";''')
    joined_df = pd.DataFrame(data = output.all(), columns = output.keys())

In [13]:
joined_df

,State,Total_SqMi,Total_SqKm,LandArea_SqMi,LandArea_SqKm,blueLaws,WeedLegalStatus,MedicinalWeed,WeedDecriminalized,sum_interstate_distance,sum_highway_distance,sum_other_distance
0,Alabama,52420,135767,50645,131171,Varies by County,Mixed,Yes,No,36.236478,84.315171,2055.135931
1,Alaska,665384,1723337,570641,1477953,No blue laws,Fully Legal,Yes,Yes,NaN,NaN,NaN
2,Arizona,113990,295234,113594,294207,No blue laws,Fully Legal,Yes,Yes,39.706072,37.897351,1519.798416
3,Arkansas,53179,137732,52035,134771,Varies by County,Mixed,Yes,No,24.131938,86.258683,2121.400732
4,California,163695,423967,155779,403466,No blue laws,Fully Legal,Yes,Yes,81.544651,46.537062,4664.484637
5,Colorado,104094,269601,103642,268431,No vehicle sales,Fully Legal,Yes,Yes,34.757985,105.259867,2022.606169
6,Connecticut,5543,14357,4842,12542,No blue laws,Mixed,Yes,Yes,11.909521,11.266595,471.569466
7,Delaware,2489,6446,1949,5047,No blue laws,Mixed,Yes,Yes,1.390199,7.388483,207.438202
8,Florida,65758,170312,53625,138887,Varies by County,Mixed,Yes,No,46.610764,111.016011,3012.123448
9,Georgia,59425,153910,57513,148959,Varies by County,Mixed,CBD Oil Only,No,39.589953,124.396675,2834.434202


In [14]:
# with engine.connect() as con:
#     output = con.execution_options(stream_results = True).execute(
#         '''SELECT pop."NAME", 
#         CAST(SUM(pop."CENSUS2010POP") AS INTEGER) AS "2010Census", 
#         CAST(SUM(pop."POPESTIMATE2019") AS INTEGER) AS "2019Estimate"
#         FROM "state_population_data_july2020" AS pop
#         GROUP BY pop."NAME"
#         ORDER BY pop."NAME";''')
#     sup = output.all()

# Push to AWS

In [15]:
joined_df.to_sql(name = 'DatabaseJoin_Deliverable', con = engine, if_exists='replace', index=False)


# Confirm Joined Dataset Exists

In [17]:
with engine.connect() as con:
    checking = con.execution_options(stream_results = True).execute(
        '''
        SELECT * 
        FROM "DatabaseJoin_Deliverable"''')
    to_check = pd.DataFrame(data = checking.all(), columns = checking.keys())

In [18]:
to_check

,State,Total_SqMi,Total_SqKm,LandArea_SqMi,LandArea_SqKm,blueLaws,WeedLegalStatus,MedicinalWeed,WeedDecriminalized,sum_interstate_distance,sum_highway_distance,sum_other_distance
0,Alabama,52420,135767,50645,131171,Varies by County,Mixed,Yes,No,36.236478,84.315171,2055.135931
1,Alaska,665384,1723337,570641,1477953,No blue laws,Fully Legal,Yes,Yes,NaN,NaN,NaN
2,Arizona,113990,295234,113594,294207,No blue laws,Fully Legal,Yes,Yes,39.706072,37.897351,1519.798416
3,Arkansas,53179,137732,52035,134771,Varies by County,Mixed,Yes,No,24.131938,86.258683,2121.400732
4,California,163695,423967,155779,403466,No blue laws,Fully Legal,Yes,Yes,81.544651,46.537062,4664.484637
5,Colorado,104094,269601,103642,268431,No vehicle sales,Fully Legal,Yes,Yes,34.757985,105.259867,2022.606169
6,Connecticut,5543,14357,4842,12542,No blue laws,Mixed,Yes,Yes,11.909521,11.266595,471.569466
7,Delaware,2489,6446,1949,5047,No blue laws,Mixed,Yes,Yes,1.390199,7.388483,207.438202
8,Florida,65758,170312,53625,138887,Varies by County,Mixed,Yes,No,46.610764,111.016011,3012.123448
9,Georgia,59425,153910,57513,148959,Varies by County,Mixed,CBD Oil Only,No,39.589953,124.396675,2834.434202
